In [1]:
import pandas as pd
import numpy as np
from functools import reduce


In [2]:
data_original = pd.read_csv("climate_change_download_0.csv")

In [3]:
print("Shape of the original data set:", data_original.shape)
print("Available columns:", data_original.columns)
print("Column data types:\n", data_original.dtypes)
print("Overview of the first 5 rows:\n", data_original.head())
print("Descriptive statistics:\n", data_original.describe())


Shape of the original data set: (13512, 28)
Available columns: Index(['Country code', 'Country name', 'Series code', 'Series name', 'SCALE',
       'Decimals', '1990', '1991', '1992', '1993', '1994', '1995', '1996',
       '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005',
       '2006', '2007', '2008', '2009', '2010', '2011'],
      dtype='object')
Column data types:
 Country code    object
Country name    object
Series code     object
Series name     object
SCALE           object
Decimals        object
1990            object
1991            object
1992            object
1993            object
1994            object
1995            object
1996            object
1997            object
1998            object
1999            object
2000            object
2001            object
2002            object
2003            object
2004            object
2005            object
2006            object
2007            object
2008            object
2009            object
2010     

In [4]:
print("Series name:\n", data_original['Series name'].unique())
print("Series code:\n", data_original['Series code'].unique())
print("SCALE:\n", data_original['SCALE'].unique())
print("Decimals:\n", data_original['Decimals'].unique())

Series name:
 ['Land area below 5m (% of land area)'
 'Agricultural land under irrigation (% of total ag. land)'
 'Cereal yield (kg per hectare)'
 'Foreign direct investment, net inflows (% of GDP)'
 'Access to electricity (% of total population)'
 'Energy use per units of GDP (kg oil eq./$1,000 of 2005 PPP $)'
 'Energy use per capita (kilograms of oil equivalent)'
 'CO2 emissions, total (KtCO2)' 'CO2 emissions per capita (metric tons)'
 'CO2 emissions per units of GDP (kg/$1,000 of 2005 PPP $)'
 'Other GHG emissions, total (KtCO2e)'
 'Methane (CH4) emissions, total (KtCO2e)'
 'Nitrous oxide (N2O) emissions, total (KtCO2e)'
 'Annex-I emissions reduction target'
 'Disaster risk reduction progress score (1-5 scale; 5=best)'
 'GHG net emissions/removals by LUCF (MtCO2e)'
 'Hosted Clean Development Mechanism (CDM) projects'
 'Hosted Joint Implementation (JI) projects'
 'Average annual precipitation (1961-1990, mm)'
 'Issued Certified Emission Reductions (CERs) from CDM (thousands)'
 'Issue

In [5]:
data_clean = data_original.copy()
print("Original number of rows:", data_clean.shape[0])

data_clean = data_clean[(data_clean['SCALE'] != 'Text') & (data_clean['Decimals'] != 'Text')]
print("Current number of rows after cleaning:", data_clean.shape[0])


Original number of rows: 13512
Current number of rows after cleaning: 10017


In [6]:
data_clean = data_clean.drop(['Country name', 'Series code', 'SCALE', 'Decimals'], axis=1)
print("Remaining columns:", data_clean.shape[1])
data_clean.iloc[:, 2:] = data_clean.iloc[:, 2:].replace({'': np.nan, '..': np.nan})
data_clean2 = data_clean.copy()
data_clean2.iloc[:, 2:] = data_clean2.iloc[:, 2:].apply(pd.to_numeric, errors='coerce')


Remaining columns: 24


In [7]:
chosen_vars = {
    'Cereal yield (kg per hectare)': 'cereal_yield',
    'Foreign direct investment, net inflows (% of GDP)': 'fdi_perc_gdp',
    'Access to electricity (% of total population)': 'elec_access_perc',
    'Energy use per units of GDP (kg oil eq./$1,000 of 2005 PPP $)': 'en_per_gdp',
    'Energy use per capita (kilograms of oil equivalent)': 'en_per_cap',
    'CO2 emissions, total (KtCO2)': 'co2_ttl',
    'CO2 emissions per capita (metric tons)': 'co2_per_cap',
    'CO2 emissions per units of GDP (kg/$1,000 of 2005 PPP $)': 'co2_per_gdp',
    'Other GHG emissions, total (KtCO2e)': 'other_ghg_ttl',
    'Methane (CH4) emissions, total (KtCO2e)': 'ch4_ttl',
    'Nitrous oxide (N2O) emissions, total (KtCO2e)': 'n2o_ttl',
    'Droughts, floods, extreme temps (% pop. avg. 1990-2009)': 'nat_emerg',
    'Population in urban agglomerations >1million (%)': 'pop_urb_aggl_perc',
    'Nationally terrestrial protected areas (% of total land area)': 'prot_area_perc',
    'GDP ($)': 'gdp',
    'GNI per capita (Atlas $)': 'gni_per_cap',
    'Under-five mortality rate (per 1,000)': 'under_5_mort_rate',
    'Population growth (annual %)': 'pop_growth_perc',
    'Population': 'pop',
    'Urban population growth (annual %)': 'urb_pop_growth_perc',
    'Urban population': 'urb_pop'
}

data_clean2['Series name'] = data_clean2['Series name'].replace(to_replace=chosen_vars)

chosen_cols = list(chosen_vars.values())
frame_list = []

In [8]:
for variable in chosen_cols:
    frame = data_clean2[data_clean2['Series name'] == variable]
    frame = frame.melt(id_vars=['Country code', 'Series name'], var_name='year', value_name=variable)
    frame = frame.rename(columns={'Country code': 'country'}).drop(columns=['Series name'])
    frame_list.append(frame)

all_vars = reduce(lambda left, right: pd.merge(left, right, on=['country', 'year'], how='outer'), frame_list)

In [9]:
print("Missing values per year:")
years_count_missing = all_vars.groupby("year").apply(lambda x: x.isnull().sum().sum()).sort_values()
print(years_count_missing)


Missing values per year:
year
2005    1189
2000    1273
1995    1317
1990    1427
2007    1631
2006    1633
2004    1646
2008    1708
2003    1714
2002    1715
2001    1718
1999    1729
1998    1739
1997    1746
1996    1756
1994    1781
1993    1792
1992    1810
1991    1921
2009    2078
2010    3038
2011    4893
dtype: int64


/tmp/ipython-input-9-4147011951.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  years_count_missing = all_vars.groupby("year").apply(lambda x: x.isnull().sum().sum()).sort_values()


In [10]:
all_vars_clean = all_vars[(all_vars['year'] >= '1991') & (all_vars['year'] <= '2000')]

country_missing = all_vars_clean.groupby('country').apply(lambda x: x.isnull().sum().sum())
countries_filter = country_missing[country_missing < 90].index.tolist()

all_vars_clean = all_vars_clean[all_vars_clean['country'].isin(countries_filter)]


/tmp/ipython-input-10-28648956.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  country_missing = all_vars_clean.groupby('country').apply(lambda x: x.isnull().sum().sum())


In [12]:
from google.colab import files

cols_to_drop = all_vars_clean.columns[all_vars_clean.isnull().sum() > 20]
all_vars_clean2 = all_vars_clean.drop(columns=cols_to_drop)


all_vars_clean3 = all_vars_clean2.dropna()

print("Final shape of the cleaned dataset:", all_vars_clean3.shape)
all_vars_clean3.to_csv('data_cleaned.csv', index=False)
files.download('data_cleaned.csv')

Final shape of the cleaned dataset: (1772, 9)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>